## 3a. Creating the Dataset

The goal of this section is to create a dataset of labeled squares:

<p>
    <img src="images/knight.png" title="Example datapoint"/>
    <br>
    <em>Label = 'n'</em>
</p>

To accomplish this, we have two collections of board images:
1. "boards/train" - Boards from Chess.com with a variety of piece styles and board colors. Collected from [kaggle.com/koryakinp/chess-positions](https://kaggle.com/koryakinp/chess-positions)
2. "boards/test" - Boards from Lichess that were collected by myself.

Each image has its FEN as the file name. First, we need a way to convert a FEN into an array of 64 labels:

In [1]:
def decode_fen(fen):
    labels = []
    fen = fen.split()[0]
    fen = fen.split(".")[0]
    rows = fen.split("-")
    for r in rows:
        for char in r:
            if char.isdigit():
                labels += ["_"] * int(char)
            else:
                labels.append(char)
    return labels

Now we're ready to build our dataset:

In [2]:
import numpy as np
from glob import glob
from PIL import Image
from comp_vision import split_image

def create_dataset(dir_name):
    squares = []
    labels = []
    # Iterate through board images
    for path in glob(dir_name + "/*.*"):
        board = Image.open(path).convert('L')
        new_squares = split_image(board)
        if not new_squares:  # Failed to detect board
            continue
        squares += [np.array(s).reshape(32,32,1) for s in new_squares]
        labels += decode_fen(path[len(dir_name)+1:])
    return np.array(squares), np.array(labels)


train_images, train_labels = create_dataset("boards/train")
test_images, test_labels = create_dataset("boards/test")
print("Size of training set:", len(train_images))
print("Size of testing set:", len(test_images))

Size of training set: 15616
Size of testing set: 1536


## 3b. Building the Model

The next step is to build a convolutional neural network (CNN) that takes in an image of a single square and outputs the piece label.

But first, we need to prepare our data. The image grayscale values need to be normalized from 0-255 to 0.0-1.0. Also, the piece labels need to be converted into one-hot encodings.

In [3]:
from tensorflow.keras.utils import to_categorical

# Normalize the pixel values
norm_train_images, norm_test_images = train_images/255., test_images/255.

# Convert string labels to ints
piece_names = ['P','N','B','R','Q','K','p','n','b','r','q','k','_']
int_train_labels = np.array([piece_names.index(label) for label in train_labels])
int_test_labels = np.array([piece_names.index(label) for label in test_labels])

# One-hot encode the labels
oh_train_labels = to_categorical(int_train_labels, num_classes=13)
oh_test_labels = to_categorical(int_test_labels, num_classes=13)

Time to build the model! It will consist of 3 convolutional layers with max pooling, followed by 2 dense layers for classification.

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten 

# Build the model
model = Sequential()

model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(32, activation='tanh'))
model.add(Dense(13, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 32)          9248      
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 32)               

## 3c. Training the Model

In [5]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(norm_train_images, oh_train_labels, epochs=10,
          validation_data=(norm_test_images, oh_test_labels),
          verbose=1)

model.save("model.h5")

Epoch 1/10
488/488 [==============================] - 3s 6ms/step - loss: 0.2831 - accuracy: 0.9322 - val_loss: 0.1389 - val_accuracy: 0.9798
Epoch 2/10
488/488 [==============================] - 2s 5ms/step - loss: 0.0562 - accuracy: 0.9876 - val_loss: 0.0665 - val_accuracy: 0.9844
Epoch 3/10
488/488 [==============================] - 3s 6ms/step - loss: 0.0270 - accuracy: 0.9945 - val_loss: 0.0523 - val_accuracy: 0.9876
Epoch 4/10
488/488 [==============================] - 3s 5ms/step - loss: 0.0153 - accuracy: 0.9967 - val_loss: 0.0260 - val_accuracy: 0.9902
Epoch 5/10
488/488 [==============================] - 2s 5ms/step - loss: 0.0085 - accuracy: 0.9985 - val_loss: 0.0293 - val_accuracy: 0.9915
Epoch 6/10
488/488 [==============================] - 2s 5ms/step - loss: 0.0048 - accuracy: 0.9995 - val_loss: 0.0344 - val_accuracy: 0.9909
Epoch 7/10
488/488 [==============================] - 2s 5ms/step - loss: 0.0040 - accuracy: 0.9995 - val_loss: 0.0301 - val_accuracy: 0.9876
Epoch 

We now have a model that is ready to use! In "predict.ipynb", we'll put it to the test.